
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [3]:
# Ajout de la colonne 'CP Ville = codePostal + espace + nomVille'

# on copie le dataframe geo pour pour laisse inchangé

geo1 = geo

geo1["CP Ville"] = geo1["Code Postal"] + " " + geo1["Commune"]

geo1.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville
Code INSEE,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,01500 AMBERIEU-EN-BUGEY
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,01330 AMBERIEUX-EN-DOMBES
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,01300 AMBLEON


In [4]:
# fonction recherche de ville
# pour cela on applique le formule pour calculer la distance entre 2 points en coordonnées cartésiennes
def ville(lat, long):
    
    # On calcule la distance entre 2 points pour chaque ligne du dataframe
    distance_lat = geo1['Latitude'].apply(lambda x: (x-lat)**2)
    distance_long = geo1['Longitude'].apply(lambda y : (y-long)**2)
    dist = np.sqrt(distance_lat + distance_long)
    
    #On renvoie le nom la commune avec la distance minimale
    return geo.loc[dist.idxmin(), 'Commune']
    
ville(46.153426, 4.926114) # Exemple de test

"L'ABERGEMENT-CLEMENCIAT"

In [5]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [6]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'BOSJEAN'

In [7]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [8]:
# Test de la librairie json

import json

y = geo1.loc["01001", "geo_shape"]

print(y)

d = json.loads(y)

d

{"type": "Polygon", "coordinates": [[[4.92627342692396, 46.1200517234555], [4.92199538031264, 46.12069170563269], [4.921996260482057, 46.12663667307488], [4.90936269187648, 46.129164756530656], [4.916016986429157, 46.14461766984426], [4.903080786126749, 46.157977074275074], [4.904571300671208, 46.16096059498421], [4.910603091203733, 46.180432597608316], [4.914119126867876, 46.18271538441311], [4.932895432675545, 46.18304616431434], [4.938729478686701, 46.17383114520867], [4.933038591703602, 46.16429901579509], [4.958286230120842, 46.153246666893935], [4.943330383388636, 46.145964148557894], [4.939154593056283, 46.12938048470421], [4.931056072485923, 46.12087123131487], [4.92627342692396, 46.1200517234555]]]}


{'type': 'Polygon',
 'coordinates': [[[4.92627342692396, 46.1200517234555],
   [4.92199538031264, 46.12069170563269],
   [4.921996260482057, 46.12663667307488],
   [4.90936269187648, 46.129164756530656],
   [4.916016986429157, 46.14461766984426],
   [4.903080786126749, 46.157977074275074],
   [4.904571300671208, 46.16096059498421],
   [4.910603091203733, 46.180432597608316],
   [4.914119126867876, 46.18271538441311],
   [4.932895432675545, 46.18304616431434],
   [4.938729478686701, 46.17383114520867],
   [4.933038591703602, 46.16429901579509],
   [4.958286230120842, 46.153246666893935],
   [4.943330383388636, 46.145964148557894],
   [4.939154593056283, 46.12938048470421],
   [4.931056072485923, 46.12087123131487],
   [4.92627342692396, 46.1200517234555]]]}

In [9]:
# On utilise la librairie Json pour parser la colonne "geo_shape"

import json

geo2 = geo1["geo_shape"].apply(lambda x: json.loads(x))
geo1["shape_type"] = geo2.apply(lambda x: x["type"])
geo1["shape_coordinates"] = geo2.apply(lambda x: x["coordinates"])

geo1[0:10]

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,shape_type,shape_coordinates
Code INSEE,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT,Polygon,"[[[4.92627342692396, 46.1200517234555], [4.921..."
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY,Polygon,"[[[5.430089291292173, 45.98275997279625], [5.4..."
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,01500 AMBERIEU-EN-BUGEY,Polygon,"[[[5.386190824025674, 45.930927620632], [5.374..."
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,01330 AMBERIEUX-EN-DOMBES,Polygon,"[[[4.895580369408855, 45.97991618124606], [4.8..."
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,01300 AMBLEON,Polygon,"[[[5.614854625053272, 45.734638855121595], [5...."
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,3359.0,2.3,"46.0055913782, 5.35760660735","{""type"": ""Polygon"", ""coordinates"": [[[5.413533...",46.005591,5.357607,01500 AMBRONAY,Polygon,"[[[5.413533405897647, 45.97889779714948], [5.3..."
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,518.0,0.7,"45.9367134524, 5.3328092349","{""type"": ""Polygon"", ""coordinates"": [[[5.321986...",45.936713,5.332809,01500 AMBUTRIX,Polygon,"[[[5.321986043355919, 45.92234494663176], [5.3..."
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,696.0,0.3,"45.7873565333, 5.65788307924","{""type"": ""Polygon"", ""coordinates"": [[[5.656393...",45.787357,5.657883,01300 ANDERT-ET-CONDON,Polygon,"[[[5.65639346945457, 45.76021095279123], [5.64..."
01010,01350,ANGLEFORT,AIN,RHONE-ALPES,Commune simple,706.0,2948.0,1.0,"45.9093715116, 5.79516005674","{""type"": ""Polygon"", ""coordinates"": [[[5.815154...",45.909372,5.795160,01350 ANGLEFORT,Polygon,"[[[5.815154073062683, 45.877200783231174], [5...."


In [10]:
# On décompte des valeurs accédées avec la clé "type"
geo1["shape_type"].value_counts()

Polygon         36670
MultiPolygon       72
Name: shape_type, dtype: int64

In [11]:
# On décompte les longueurs des listes accédées avec la clé "coordinates".
geo1["shape_coordinates"].apply(len).value_counts()

1    36660
2       80
4        1
3        1
Name: shape_coordinates, dtype: int64

In [12]:
#Quelle commune est la plus complexe géométriquement ? (1/2)

# On cherche la commune la plus complexe géométriquement (i.e la commmune qui possède le plus de coordonnées)
# On distingue deux types de clé pour la forme géométrique d'une commune ("MultiPolygon" et "Polygon")
# Les communes les plus complexes géométriquement sont les communes qui sont de types 

geo_multipolygone = geo1.loc[geo1["shape_type"] == "MultiPolygon"]

geo_multipolygone["shape_coordinates"].apply(len).value_counts()

2    70
4     1
3     1
Name: shape_coordinates, dtype: int64

In [13]:
#Quelle commune est la plus complexe géométriquement ? (2/2)

# On remarque qu'il y a une commune qui possède la plus grande taille de l'attribut "shape_coordinates" qui est de 4
# On cherche donc à afficher le nom de cette commune

geo_multipolygone.loc[geo_multipolygone["shape_coordinates"].apply(len) == 4, "Commune"]

Code INSEE
83069    HYERES
Name: Commune, dtype: object

In [14]:
# On cherche la seconde commune la plus complexe géométriquement ?
# La seconde commune la plus complexe géométriquement correspond à la commune dont la tailles de l'attribut "shape_coordinates" est de 3
# On cherche donc à afficher le nom de cette commune

geo_multipolygone.loc[geo_multipolygone["shape_coordinates"].apply(len) == 3, "Commune"]

Code INSEE
27701    VAL-DE-REUIL
Name: Commune, dtype: object

In [15]:
# Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?

commune_polygon_type = geo1.loc[(geo1["shape_type"] == "Polygon") & (geo1["shape_coordinates"].apply(len) == 2)]
commune_polygon_type

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,shape_type,shape_coordinates
Code INSEE,,,,,,,,,,,,,,,
02232,02600,COYOLLES,AISNE,PICARDIE,Commune simple,126.0,2489.0,0.4,"49.2133089693, 3.01306173751","{""type"": ""Polygon"", ""coordinates"": [[[3.037897...",49.213309,3.013062,02600 COYOLLES,Polygon,"[[[3.037897560658703, 49.16921270775551], [3.0..."
05052,05600,EYGLIERS,HAUTES-ALPES,PROVENCE-ALPES-COTE D'AZUR,Commune simple,1615.0,2987.0,0.7,"44.6925244966, 6.66688741371","{""type"": ""Polygon"", ""coordinates"": [[[6.719912...",44.692524,6.666887,05600 EYGLIERS,Polygon,"[[[6.719912606601575, 44.70392733592357], [6.7..."
09042,09240,LA BASTIDE-DE-SEROU,ARIEGE,MIDI-PYRENEES,Chef-lieu canton,479.0,4138.0,1.0,"43.0242795037, 1.42677885373","{""type"": ""Polygon"", ""coordinates"": [[[1.423960...",43.024280,1.426779,09240 LA BASTIDE-DE-SEROU,Polygon,"[[[1.423960819701936, 42.99550854650971], [1.4..."
22209,22650,PLOUBALAY,COTES-D'ARMOR,BRETAGNE,Chef-lieu canton,36.0,3600.0,2.7,"48.5596688192, -2.12838362694","{""type"": ""Polygon"", ""coordinates"": [[[-2.11237...",48.559669,-2.128384,22650 PLOUBALAY,Polygon,"[[[-2.11237709725211, 48.52896113651634], [-2...."
2A272,20100,SARTENE,CORSE-DU-SUD,CORSE,Sous-préfecture,190.0,20182.0,3.1,"41.5718443328, 8.93538107497","{""type"": ""Polygon"", ""coordinates"": [[[8.940487...",41.571844,8.935381,20100 SARTENE,Polygon,"[[[8.940487655060174, 41.490088793265336], [8...."
2B049,20214,CALENZANA,HAUTE-CORSE,CORSE,Chef-lieu canton,596.0,18465.0,2.1,"42.4784267166, 8.80369225001","{""type"": ""Polygon"", ""coordinates"": [[[8.911220...",42.478427,8.803692,20214 CALENZANA,Polygon,"[[[8.911220447280732, 42.41019895400797], [8.8..."
48080,48300,LANGOGNE,LOZERE,LANGUEDOC-ROUSSILLON,Chef-lieu canton,973.0,3124.0,3.1,"44.7237405653, 3.83363110341","{""type"": ""Polygon"", ""coordinates"": [[[3.850531...",44.723741,3.833631,48300 LANGOGNE,Polygon,"[[[3.850531831651621, 44.68745054452225], [3.8..."
67486,67920,SUNDHOUSE,BAS-RHIN,ALSACE,Commune simple,166.0,1590.0,1.5,"48.2550723499, 7.63031906582","{""type"": ""Polygon"", ""coordinates"": [[[7.594939...",48.255072,7.630319,67920 SUNDHOUSE,Polygon,"[[[7.594939251325704, 48.23228567615893], [7.5..."
68078,68420,EGUISHEIM,HAUT-RHIN,ALSACE,Commune simple,320.0,1400.0,1.6,"48.0371183049, 7.30052876111","{""type"": ""Polygon"", ""coordinates"": [[[7.281156...",48.037118,7.300529,68420 EGUISHEIM,Polygon,"[[[7.281156117554148, 48.02598189605906], [7.2..."


In [16]:
# on importe la classe Polygon de la librairie python shapely.geometry
from shapely.geometry import Polygon

# On cherche à vérifier que pour les communes de types "Polygon" le 1er polygône contient bien le 2ème polygône

commune_polygon_type_isEnclave = commune_polygon_type

commune_polygon_type_isEnclave["isEnclave"] = commune_polygon_type_isEnclave["shape_coordinates"].apply(lambda x: Polygon(x[0]).contains(Polygon(x[1])))

commune_polygon_type_isEnclave

<ipython-input-16-2b30fbfc2111>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commune_polygon_type_isEnclave["isEnclave"] = commune_polygon_type_isEnclave["shape_coordinates"].apply(lambda x: Polygon(x[0]).contains(Polygon(x[1])))


,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,shape_type,shape_coordinates,isEnclave
Code INSEE,,,,,,,,,,,,,,,,
02232,02600,COYOLLES,AISNE,PICARDIE,Commune simple,126.0,2489.0,0.4,"49.2133089693, 3.01306173751","{""type"": ""Polygon"", ""coordinates"": [[[3.037897...",49.213309,3.013062,02600 COYOLLES,Polygon,"[[[3.037897560658703, 49.16921270775551], [3.0...",True
05052,05600,EYGLIERS,HAUTES-ALPES,PROVENCE-ALPES-COTE D'AZUR,Commune simple,1615.0,2987.0,0.7,"44.6925244966, 6.66688741371","{""type"": ""Polygon"", ""coordinates"": [[[6.719912...",44.692524,6.666887,05600 EYGLIERS,Polygon,"[[[6.719912606601575, 44.70392733592357], [6.7...",True
09042,09240,LA BASTIDE-DE-SEROU,ARIEGE,MIDI-PYRENEES,Chef-lieu canton,479.0,4138.0,1.0,"43.0242795037, 1.42677885373","{""type"": ""Polygon"", ""coordinates"": [[[1.423960...",43.024280,1.426779,09240 LA BASTIDE-DE-SEROU,Polygon,"[[[1.423960819701936, 42.99550854650971], [1.4...",True
22209,22650,PLOUBALAY,COTES-D'ARMOR,BRETAGNE,Chef-lieu canton,36.0,3600.0,2.7,"48.5596688192, -2.12838362694","{""type"": ""Polygon"", ""coordinates"": [[[-2.11237...",48.559669,-2.128384,22650 PLOUBALAY,Polygon,"[[[-2.11237709725211, 48.52896113651634], [-2....",True
2A272,20100,SARTENE,CORSE-DU-SUD,CORSE,Sous-préfecture,190.0,20182.0,3.1,"41.5718443328, 8.93538107497","{""type"": ""Polygon"", ""coordinates"": [[[8.940487...",41.571844,8.935381,20100 SARTENE,Polygon,"[[[8.940487655060174, 41.490088793265336], [8....",True
2B049,20214,CALENZANA,HAUTE-CORSE,CORSE,Chef-lieu canton,596.0,18465.0,2.1,"42.4784267166, 8.80369225001","{""type"": ""Polygon"", ""coordinates"": [[[8.911220...",42.478427,8.803692,20214 CALENZANA,Polygon,"[[[8.911220447280732, 42.41019895400797], [8.8...",True
48080,48300,LANGOGNE,LOZERE,LANGUEDOC-ROUSSILLON,Chef-lieu canton,973.0,3124.0,3.1,"44.7237405653, 3.83363110341","{""type"": ""Polygon"", ""coordinates"": [[[3.850531...",44.723741,3.833631,48300 LANGOGNE,Polygon,"[[[3.850531831651621, 44.68745054452225], [3.8...",True
67486,67920,SUNDHOUSE,BAS-RHIN,ALSACE,Commune simple,166.0,1590.0,1.5,"48.2550723499, 7.63031906582","{""type"": ""Polygon"", ""coordinates"": [[[7.594939...",48.255072,7.630319,67920 SUNDHOUSE,Polygon,"[[[7.594939251325704, 48.23228567615893], [7.5...",True
68078,68420,EGUISHEIM,HAUT-RHIN,ALSACE,Commune simple,320.0,1400.0,1.6,"48.0371183049, 7.30052876111","{""type"": ""Polygon"", ""coordinates"": [[[7.281156...",48.037118,7.300529,68420 EGUISHEIM,Polygon,"[[[7.281156117554148, 48.02598189605906], [7.2...",True
